### Descargar datos via AWS S3

Para saber que PATH y ROW escoger para encontrar nuestra zona de estudio, se usará el siguiente enlace para encontrar las variables para COROPUNA:
- https://landsat.usgs.gov/landsat_acq#convertPathRow
- {PATH 4, ROW 71}

A la hora de acceder a la información del bucket, esta vendrá proporcionada de la siguiente manera: /SENSOR_ID/01/PATH/ROW/SCENE_ID/
- SENSOR_ID : identificador del satélite e instrumento
- 01 : indicador de datos por parte de Landsat Collection 1
- PATH : ruta WRS escogida
- ROW : fila WRS escogida
- SCENE_ID : ID único de la escena

http://geologyandpython.com/get-landsat-8.html


#### Descarga del catálogo Landsat 8

In [1]:
import numpy as np
from osgeo import gdal
import glob
import json
import Utils as ut
import VectorRast as vr
import IoRast   as rose
from os.path import join
import TempSurface as tm
import AtmosCorrection as ac
import pandas as pd


ROW = 32
PATH = 202

# Abrimos el fichero de parámetros de configuración del usuario
with open ('config.json', 'r') as file: config = json.load(file)


# Descargamos el catálogo 
# 'http://landsat-pds.s3.amazonaws.com/c1/L8/scene_list.gz'
s3_scenes = pd.read_csv('Catalogo/scene_list.gz', compression='gzip')

# Escogemos el path y row de COROPUNA, escogemos aquellas imágenes de calidad Tier 1, calibradas y referenciadas
s3_scenes = s3_scenes[(s3_scenes.path == PATH) & (s3_scenes.row == ROW) & (~s3_scenes.productId.str.contains('_T2')) & (~s3_scenes.productId.str.contains('_RT'))]

# Si existe varias tomas de una misma escena, entonces escogemos la que menor cobertura nubosa contenga y ordenamos la tabla por fecha de adquisición de los datos
s3_scenes = s3_scenes.sort_values('cloudCover').groupby(s3_scenes.productId).first().sort_values('acquisitionDate')

print("COVER")
# Escogemos aquellas escenas cuya cobertura nubosa sea inferior a un 70%
s3_scenes = s3_scenes[s3_scenes.cloudCover < 70]

print("FECHA")
# Escogemos todas las escenas de 2014 hacia delante
s3_scenes = s3_scenes[s3_scenes.acquisitionDate  > '2014-01-01']

COVER
FECHA


#### Procesamiento en descarga

In [2]:
import numpy as np
from osgeo import gdal
import glob
import json
import Utils as ut
import VectorRast as vr
import IoRast   as rose
from os.path import join
import TempSurface as tm
import AtmosCorrection as ac
from time import time

gdal.SetConfigOption('CPL_VSIL_CURL_ALLOWED_EXTENSIONS', 'tif')
gdal.SetConfigOption('VSI_CACHE', 'TRUE')
gdal.SetConfigOption('GDAL_DISABLE_READDIR_ON_OPEN', 'TRUE')
gdal.SetConfigOption('CPL_VSIL_CURL_CHUNK_SIZE', '32768')
gdal.SetConfigOption('CPL_VSIL_CURL_CACHE_SIZE', '655360')
gdal.SetConfigOption('CPL_DEBUG', 'OFF')
gdal.SetConfigOption('AWS_NO_SIGN_REQUEST', 'YES')

i, ms = 14, [9, 10, 11, 12, 1, 2, 3]

while i < len(s3_scenes):
    if int(str(s3_scenes.iloc[i].acquisitionDate).split('-')[1]) in ms:
        
 
        print(f'Procesando archivo número {i}')

        print('Fecha adquisicion:', {s3_scenes.iloc[i]['acquisitionDate']})
        print('% Cobertura nubosa', {s3_scenes.iloc[i]['cloudCover']})
        start_time = time()
        
        # Creamos las url para cada banda vsicurl
        url = '/'.join(s3_scenes.iloc[i]['download_url'].split('/')[0:-1])
        blue, green, red, nir, swir1, tir1 = [f'/vsicurl/{url}/{s3_scenes.iloc[i].productId}_B{j}.TIF' for j in [2, 3, 4, 5, 6, 10]]

        # Creamos la url para el archivo MTL
        MTL = f'{url}/{s3_scenes.iloc[i].productId}_MTL.txt'
        
        # Cargamos contenido del archivo MTL en memoria
        params = rose.readOnMTL(MTL)
        
        # Introducimos nuevos parametros que se quitarán posteriormente
        config['date'] = params['DATE_ACQUIRED']
        config['mrs'] = 1 
        config['aoi'] = None #(2000, 3000, 4000, 2000) #(3000, 2000, 2000, 1000) COROPUNA

        # Cargamos las imágenes en memoria
        r1, tir1   = rose.loadRasterImage(tir1 , config['aoi'])
        r2, blue   = rose.loadRasterImage(blue , config['aoi'])
        r3, green  = rose.loadRasterImage(green, config['aoi'])
        r4, red    = rose.loadRasterImage(red  , config['aoi'])
        r5, nir    = rose.loadRasterImage(nir  , config['aoi'])
        r6, swir1  = rose.loadRasterImage(swir1, config['aoi'])

        elapsed_time = time() - start_time
        print("Tiempo en descargar y cargar bandas: %0.10f seconds." % elapsed_time)
      
        if config['aoi'] is None:
            config['aoi'] = (0, 0, 0, 0)
 
        '''# Corregimos atmosfericamente
        blue       = ac.atmosCorrection(2, params, blue , config['mode'])
        green      = ac.atmosCorrection(3, params, green, config['mode'])
        swir1      = ac.atmosCorrection(6, params, swir1, config['mode'])


        # Generamos NDSI para calcular mascara de nieve
        NDSI       = ut.calcNormIndex([swir1, green])

        # Conseguimos la temperatura en superficie y la pasamos a grados celsius
        res        = tm.tir2Temp(red, nir, tir1, params)
        res        = res - 273.15

        # Definimos que existe presencia de nieve cuando el NDSI > 0.1 y la temperatura es mayor a -20 grados
        presencia  = (NDSI >= 0.20) & (res > -20) #-20

        # Como el NDSI tambien coge masas de agua, una forma fácil de eliminarlas es tomando como máscara, aquellas zona de temperaturas mayores a 5 grados.
        presencia &= (res < 5)

        # A su vez, se tendrá en cuenta una máscara para eliminar nubes más densas o de mayor altura, haciendo uso de la banda swir del instrumento
        presencia &= (ut.norm(blue) < 0.8) & (ut.norm(swir1) < 0.2)

        # Tras ello, añadimos las coberturas claras de nieve a la solucion
        presencia |= (NDSI >= 0.4)

        NDSI[~presencia] = config['mini']

        print('Almacenando máscara...')
        out = join(config['PathMASK'],s3_scenes.iloc[i]['productId'] + '.tif' )
        rose.saveBandAsTiff(out, r6, NDSI)'''
        
        print('Vectorizando...')
        
        #vr.generateShapes(NDSI, r6, config)
        
        print('Finalizado')
        break
        
        
    i += 1
# https://lucid.app/lucidchart/f7c340f6-ef6e-42e4-8bb1-a9c7f6eec8f6/edit?beaconFlowId=925647224A203237&page=0_0#

Procesando archivo número 14
Fecha adquisicion: {'2014-10-31 11:01:55.945972'}
% Cobertura nubosa {1.38}
Tiempo en descargar y cargar bandas: 76.3153791428 seconds.
Vectorizando...
Finalizado


In [6]:
green.min()

0

In [ ]:
# recortar -> https://gidahatari.com/ih-es/como-cortar-multiples-bandas-de-una-imagen-landsat-8-con-python-y-gdal-tutorial

### Recortar haciendo uso de un shapefile

In [27]:
import numpy as np
from osgeo import gdal
import glob
import json
import Utils as ut
import VectorRast as vr
import IoRast   as rose
from os.path import join
import TempSurface as tm
import AtmosCorrection as ac
import pandas as pd


# Abrimos el fichero de parámetros de configuración del usuario
with open ('config.json', 'r') as file: config = json.load(file)

inn = r'D:\Mask\imgs'
shp = r'D:\Mask\shp\AOI.shp'
out = r'D:\Mask\crops'

tifs = glob.glob(join(inn, '*.tif'))


for band in tifs:
    print(band[:-4]+'_crop'+band[-4:])
    options = gdal.WarpOptions(cutlineDSName=shp,cropToCutline=True)
    outBand = gdal.Warp(srcDSOrSrcDSTab=band,
                        destNameOrDestDS=band[:-4]+'_crop'+band[-4:],
                        options=options)
    outBand= None
   


D:\Mask\imgs\LC08_L1TP_202032_20140201_20170426_01_T1_crop.tif
D:\Mask\imgs\LC08_L1TP_202032_20140305_20180527_01_T1_crop.tif
D:\Mask\imgs\LC08_L1TP_202032_20140913_20170419_01_T1_crop.tif
D:\Mask\imgs\LC08_L1TP_202032_20140929_20170419_01_T1_crop.tif
D:\Mask\imgs\LC08_L1TP_202032_20141031_20170418_01_T1_crop.tif
D:\Mask\imgs\LC08_L1TP_202032_20141116_20170417_01_T1_crop.tif
D:\Mask\imgs\LC08_L1TP_202032_20141202_20170416_01_T1_crop.tif
D:\Mask\imgs\LC08_L1TP_202032_20141218_20170416_01_T1_crop.tif
D:\Mask\imgs\LC08_L1TP_202032_20150103_20170415_01_T1_crop.tif
D:\Mask\imgs\LC08_L1TP_202032_20150119_20170413_01_T1_crop.tif
D:\Mask\imgs\LC08_L1TP_202032_20150204_20170413_01_T1_crop.tif
D:\Mask\imgs\LC08_L1TP_202032_20150308_20170412_01_T1_crop.tif
D:\Mask\imgs\LC08_L1TP_202032_20151002_20170403_01_T1_crop.tif
D:\Mask\imgs\LC08_L1TP_202032_20151119_20170402_01_T1_crop.tif
D:\Mask\imgs\LC08_L1TP_202032_20151205_20170401_01_T1_crop.tif
D:\Mask\imgs\LC08_L1TP_202032_20151221_20170331_01_T1_c

### Sacar estadísticos por año

In [2]:
import numpy as np
import IoRast   as rose
import matplotlib.pyplot as plt
import glob
from os.path import join


# std por cada año
for folder in glob.glob(join(config['PathMASK'],'imgs/*/')):
    print(folder)
    st = []
    for tif in glob.glob(join(folder, '*_crop.tif')):
        print('\t',tif)
        rt, img = rose.loadRasterImage(tif , None)
        st.append(img)
    # Una vez cargadas en memoria todas las imágenes se realiza el stack
    st = np.stack(st, axis = 2)

    rose.saveBandAsTiff(join(folder, 'mean.tif'), rt, np.nanmean(st, axis = 2))
    rose.saveBandAsTiff(join(folder, 'median.tif'), rt, np.nanmedian(st, axis = 2))
    rose.saveBandAsTiff(join(folder, 'std.tif'), rt, np.nanstd(st, axis = 2))

D:/Mask/imgs\2014\
	 D:/Mask/imgs\2014\LC08_L1TP_202032_20140201_20170426_01_T1_crop.tif
	 D:/Mask/imgs\2014\LC08_L1TP_202032_20140305_20180527_01_T1_crop.tif
	 D:/Mask/imgs\2014\LC08_L1TP_202032_20141116_20170417_01_T1_crop.tif
	 D:/Mask/imgs\2014\LC08_L1TP_202032_20141202_20170416_01_T1_crop.tif
	 D:/Mask/imgs\2014\LC08_L1TP_202032_20141218_20170416_01_T1_crop.tif
D:/Mask/imgs\2015\
	 D:/Mask/imgs\2015\LC08_L1TP_202032_20150103_20170415_01_T1_crop.tif
	 D:/Mask/imgs\2015\LC08_L1TP_202032_20150119_20170413_01_T1_crop.tif
	 D:/Mask/imgs\2015\LC08_L1TP_202032_20150308_20170412_01_T1_crop.tif
	 D:/Mask/imgs\2015\LC08_L1TP_202032_20151119_20170402_01_T1_crop.tif
	 D:/Mask/imgs\2015\LC08_L1TP_202032_20151205_20170401_01_T1_crop.tif
	 D:/Mask/imgs\2015\LC08_L1TP_202032_20151221_20170331_01_T1_crop.tif
D:/Mask/imgs\2016\
	 D:/Mask/imgs\2016\LC08_L1TP_202032_20160207_20170330_01_T1_crop.tif
	 D:/Mask/imgs\2016\LC08_L1TP_202032_20160223_20180527_01_T1_crop.tif
	 D:/Mask/imgs\2016\LC08_L1TP_202

### Sacar estadísticos por año

In [ ]:
import numpy as np
import IoRast   as rose
import matplotlib.pyplot as plt
import glob
from os.path import join


st = []
for folder in glob.glob(join(config['PathMASK'],'imgs/*/')):
    print(folder)
    for tif in glob.glob(join(folder, '*_crop.tif')):
        rt, img = rose.loadRasterImage(tif , None)
        st.append(img)
    # Una vez cargadas en memoria todas las imágenes se realiza el stack
    
st = np.stack(st, axis = 2)

rose.saveBandAsTiff(join(config['PathMASK'], 'imgs/mean.tif'), rt, np.nanmean(st, axis = 2))
rose.saveBandAsTiff(join(config['PathMASK'], 'imgs/std.tif'), rt, np.nanstd(st, axis = 2))
rose.saveBandAsTiff(join(config['PathMASK'], 'imgs/median.tif'), rt, np.nanmedian(st, axis = 2))


D:/Mask/imgs\2014\
D:/Mask/imgs\2015\
D:/Mask/imgs\2016\
D:/Mask/imgs\2017\
D:/Mask/imgs\2018\
D:/Mask/imgs\2019\
D:/Mask/imgs\2020\
D:/Mask/imgs\RAW\


In [23]:
# Mediana y STD para todas los ficheros de la mediana
st = []
for tif in glob.glob(join(config['PathMASK'],'*/mean.tif')):
    rt, img = rose.loadRasterImage(tif , None)
    st.append(img)
st = np.stack(st, axis = 2)

median = np.nanmedian(st, axis = 2)
std    = np.nanstd(st, axis = 2)
mean   = np.nanmean(st, axis = 2)

rose.saveBandAsTiff(join(config['PathMASK'], 'median_fin_mean.tif'), rt, median)
rose.saveBandAsTiff(join(config['PathMASK'], 'std_fin_mean.tif'), rt, std)
rose.saveBandAsTiff(join(config['PathMASK'], 'mean_fin_mean.tif'), rt, mean)

In [12]:
import geopandas as gpd
from shapely.geometry import mapping, Polygon, Point
from shapely.ops import cascaded_union, unary_union
import fiona
from fiona.crs import from_epsg
import glob
import Utils as ut

sc, cont = {'geometry': 'Polygon','properties': {'id': 'int', 'date': 'str', 'area':'float'}}, 0
dst = f'out/union.shp'
ut.makeFolder(f'out/') 

with fiona.open(dst, 'w', 'ESRI Shapefile',crs=from_epsg(32618), schema=sc) as c:  # creates new file to be written to
    for path in glob.glob('Shape\**\*.shp'):
        print(cont)
    # Leemos los datos del shapefile
        sh = gpd.read_file(path)
        if len(sh) > 9
        polis = sh['geometry']
        print(sh['date'])
        date = sh['date'][0]
    
    # Realizamos el fichero shapefile resultante
        for j in range(len(polis)):
            c.write({'geometry': mapping(polis[j]),'properties': {'id': cont, 'date': sh['date'][0], 'area': polis[j].area},})
            cont += 1

0
0
Series([], Name: date, dtype: object)


IndexError: index 0 is out of bounds for axis 0 with size 0